# Classification Pipeline

In [ ]:
# File manipulation
import os
from os.path import exists

# General
import numpy as np
import pandas as pd

# Sci-kit learn
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import cluster

# Tensorflow
import tensorflow as tf
from tensorflow import keras
tf.compat.v1.enable_eager_execution()  # This allows you to use placeholder in version 2.0 or higher
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

# LSTM
from tensorflow.keras.layers import LSTM, Dense

# Transformer
from tensorflow.keras.layers import Embedding, MultiHeadAttention, Dense, Input, Dropout, LayerNormalization

# N-Layer NN
from tensorflow.keras.layers import LeakyReLU, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization

# CNN
from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten, GlobalMaxPool2D, MaxPool2D, concatenate, Activation)

np.random.seed(1)

# Subfunctions

In [ ]:
def resize_img(img, img_dim):
    if type(img) == 'numpy.ndarray':
        # img is an array, retuns an image object
        rgb_image = Image.fromarray(img , 'RGB')
    else:
        # img is an image object, returns an image object
        try:
            rgb_image = img.convert('RGB')
        except AttributeError:
            rgb_image = Image.fromarray(img , 'RGB')

    # Resize image into a 64, 64, 3
    new_h, new_w = int(img_dim), int(img_dim)
    img3 = rgb_image.resize((new_w, new_h), Image.ANTIALIAS)
    w_resized, h_resized = img3.size[0], img3.size[1]
    return img3

def convert_img_a_mat(img, outpt):
    mat = np.array(img)  # Convert image to an array
    if outpt == 'mat2D':
        # Transformer l'image de 3D à 2D
        # Convert image back to a 2D array
        matout = np.mean(mat, axis=2)
    elif outpt == 'img3D': # techniquement c'est un image parce qu'il y a trois RGB channels 
        matout = mat
    return matout

def norm_mat(mat2Dor3D, norm):
    if norm == 'zero2one':
        # Normalizer l'image entre 0 et 1
        norout = mat2Dor3D/255
    elif norm == 'negone2posone':
        # Normalize the images to [-1, 1]
        norout = (mat2Dor3D - 127.5) / 127.5
    elif norm == 'non':
        norout = mat2Dor3D
    return norout

def threshold_mat(mat2D, thresh):
    # Threshold image
    val = 255/2
    if thresh == 'zero_moins_que_val':
        row, col = mat2D.shape
        mat_thresh = mat2D
        min_val = np.min(mat_thresh)
        for i in range(row):
            for j in range(col):
                if mat_thresh[i,j] < val:
                    mat_thresh[i,j] = min_val
    elif thresh == 'non':
        mat_thresh = mat2D
    return mat_thresh

def imgORmat_resize_imgORmat_CNN(img_dim, data_in, inpt='img3D', outpt='mat2D', norm='non', thresh='non'):
    if inpt == 'img3D' and outpt=='mat2D':
        img = resize_img(data_in, img_dim)
        img3D = convert_img_a_mat(img, outpt)
        out = norm_mat(img3D, norm)
    elif inpt == 'mat2D' and outpt=='mat2D':
        data_in = np.array(data_in)
        img = Image.fromarray(data_in , 'L')
        img = resize_img(img, img_dim)
        mat2D = convert_img_a_mat(img, outpt)
        out = norm_mat(mat2D, norm)
    elif inpt == 'mat2D' and outpt=='img3D':
        data_in = np.array(data_in)
        img = Image.fromarray(data_in , 'L')
        img = resize_img(img, img_dim)
        img3D = convert_img_a_mat(img, outpt)
        out = norm_mat(img3D, norm)
    elif inpt == 'img3D' and outpt=='img3D':
        img = resize_img(data_in, img_dim)
        img3D = convert_img_a_mat(img, outpt)
        out = norm_mat(img3D, norm)

    return out

In [ ]:
def permutation_importance_tensorflow(model, X_test, Y_test):

    Y_test_1D = [Y_test[i,0:1] for i in range(Y_test.shape[0])]

    # First, a baseline metric, defined by scoring,
    # Obtenez mean absolute error
    y_hat_test = model.predict(X_test, verbose=0)
    baseline_mae = np.mean(np.abs(y_hat_test - Y_test_1D))

    vals = {}
    # Shuffle each feature columns at a time
    for featcol in range(X_test.shape[2]):

        # Define a modifiable temporary variable
        temp = X_test

        # select a column
        feat_slice = temp[:,:,featcol]

        # Must flatten the matrix because np.random.permutation or 
        # np.random.shuffle don't work
        t = feat_slice.flatten()
        t_shuf = np.random.permutation(t)
        feat_slice =  np.reshape(t_shuf, (feat_slice.shape))

        # put feat_slice back into temp
        temp[:,:,featcol] = feat_slice

        y_hat_test = model.predict(temp, verbose=0)
        mae_per_col = np.mean(np.abs(y_hat_test - Y_test_1D))
        vals[featcol] = mae_per_col

    # Sort the columns from largest to smallest mae
    laquelle = sort_dict_by_value(vals, reverse = True)
    
    # Determinez le nombres des columns qui sont plus grande que le baseline_mae
    # C'est des marqueurs qui sont importants
    feat = list(laquelle.keys())
    cnt = [1 for i in range(len(feat)) if feat[i] > baseline_mae]
    cnt = np.sum(cnt)
    
    allout = list(laquelle.items())
    nout = [allout[i] for i in range(cnt)]
    marquers_important = dict(nout)
    
    return marquers_important

In [ ]:
def classify(df_test2, ynum):
    
    # Loop over each model to test
    for wm in [3, 1, 4, 0, 2, 5]:
        res_permod = []
        
        for fea in range(4):
            print('fea : ', fea)
            
            # ----------------
            # Order of which features to use in a model
            if fea == 0:
                # 1) All features
                X_cols = list(np.arange(1, df_test2.shape[1]-1, 1))
                y_cols = [df_test2.shape[1]-1]
            elif fea == 1:
                # 2) first 3 from permutation_importance
                X_cols = [list(marquers_important.items())[oo][0] for oo in range(3)]
                y_cols = [df_test2.shape[1]-1]
            elif fea == 2:
                # 3) first 2 from permutation_importance
                X_cols = [list(marquers_important.items())[oo][0] for oo in range(2)]
                y_cols = [df_test2.shape[1]-1]
            elif fea == 3:
                # 4) first feature from permutation_importance
                X_cols = [list(marquers_important.items())[0][0]]
                y_cols = [df_test2.shape[1]-1] 
            
            
            # ORIGINAL MODELS:
            if wm == 0:
                # Sequential : Support Vector Machine
                m_name = 'SVC'
                batch_size = 24
                X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols)
                model, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test = svm_batch(X_train, X_test, Y_train, Y_test, info, batch_size)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    # Stack all data over batches
                    X_test_batch = np.reshape(X_test, (info['batch_test']*info['timesteps_test'], info['feature_test']))
                    Y_test_1D_batch = np.reshape(Y_test, (info['batch_test']*info['timesteps_test'], info['n_outputs']))

                    r = permutation_importance(model, X_test_batch, Y_test_1D_batch, n_repeats=10, random_state=0, scoring='accuracy')
                    vals = dict(zip(np.arange(len(r.importances_mean)), r.importances_mean))
                    marquers_important = sort_dict_by_value(vals, reverse = True) # Sort the columns from largest to smallest mae
                # ----------------
                
            elif wm == 3:
                # Spatial (find global trends in the feature) : RandomForest - partitioned subspace
                m_name = 'RF'
                X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols)
                
                # Stack all data over batch
                X_train = np.reshape(X_train, (info['batch_train']*info['timesteps_train'], info['feature_train']))
                Y_train_1D = np.reshape(Y_train, (info['batch_train']*info['timesteps_train'], info['n_outputs']))
                X_test = np.reshape(X_test, (info['batch_test']*info['timesteps_test'], info['feature_test']))
                Y_test_1D = np.reshape(Y_test, (info['batch_test']*info['timesteps_test'], info['n_outputs']))
                
                model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_score, Y_test_score, dfs = RandomForest(X_train, X_test, Y_train_1D, Y_test_1D)

                value_pack_train = evaluation_methods(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_score, dfs)
                value_pack_test = evaluation_methods(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_score, dfs)

                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(value_pack_train, q=2)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(value_pack_test, q=3)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    r = permutation_importance(model, X_test, Y_test_1D, n_repeats=10, random_state=0, scoring='accuracy')
                    vals = dict(zip(np.arange(len(r.importances_mean)), r.importances_mean))
                    marquers_important = sort_dict_by_value(vals, reverse = True) # Sort the columns from largest to smallest mae
                # ----------------
        
        elif wm == 5:
                # Multilayer perceptron (MLP)/neural network (Deep Learning) : logistic regression NN
                m_name = 'MLP'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_multilayer_perceptron(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_multilayer_perceptron_1Dinput(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
        
        elif wm == 6:
                # Gaussian Naive Bayes
                m_name = 'GNB'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_gaussian_naive_bayes(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_gaussian_naive_bayes(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            
            # UPDATED MODELS in HAR :
            elif wm == 1:
                # Sequential : LSTM - changes within a window of points
                m_name = 'LSTM'
                model, dict_out, X_test, Y_test = run_LSTM(df_test2, X_cols, y_cols, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = [dict_out['delay_train'], dict_out['delay_test']]    
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                
            elif wm == 2:
                # Sequential : Transformer - changes between windows of points
                m_name = 'Trans'
                model, dict_out, X_test, Y_test = run_Transformer(df_test2, X_cols, y_cols, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                
            elif wm == 4:
                # Spatial (find global trends in the feature) : CNN
                m_name = 'CNN'
                model, dict_out, X_test, Y_test = run_CNN(df_test2, X_cols, y_cols, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                
            elif wm == 5:
                # Sequential & Spatial : LSTM-CNN
                m_name = 'LSTM-CNN'
                model, dict_out, X_test, Y_test = run_LSTM_CNN(df_test2, X_cols, y_cols, ynum)
                acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(dict_out, q=0)
                acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(dict_out, q=1)
                extra = np.nan
                # ----------------
                # Permutation importance of features : probe which features are most predictive
                if fea == 0:
                    marquers_important = permutation_importance_tensorflow(model, X_test, Y_test)
                # ----------------
                

            # ----------------
            # Save all data to array 
            res_permod.append([ynum, m_name, fea, X_cols, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test, marquers_important, extra])
            # ----------------
            
        # Save data matrices to file per model result :
        file_name = "res_exp_%s_%s_%s_ynum%d_%s.pkl" % (exp, ax_val, ss_val, ynum, m_name)
        open_file = open(file_name, "wb")
        pickle.dump(res_permod, open_file)
        open_file.close()
            
    return

In [ ]:
def mettez_dictout_dans_vars(dict_out, q):
    if q == 0:
        acc = dict_out['acc_train']
        prec = dict_out['prec_train']
        recall = dict_out['recall_train']
        roc_auc = dict_out['roc_auc_train']
    elif q == 1:
        acc = dict_out['acc_test']
        prec = dict_out['prec_test']
        recall = dict_out['recall_test']
        roc_auc = dict_out['roc_auc_test']
    elif q == 2:
        acc = dict_out['acc_dircalc']
        prec = dict_out['prec_dircalc']
        recall = dict_out['recall_dircalc']
        roc_auc = dict_out['rocauc_dircalc']
    elif q == 3:
        acc = dict_out['acc_dircalc']
        prec = dict_out['prec_dircalc']
        recall = dict_out['recall_dircalc']
        roc_auc = dict_out['rocauc_dircalc']
            
    return acc, prec, recall, roc_auc

In [ ]:
def df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols):

    # Ensure that the X matrix size is correct
    # df_test2 : (dp_per_sample*n_values, feature)
    all_dp, cols = df_test2.shape

    # ----------------

    # Il faut change df_test2 à : (batch, timesteps, feature)
    needed_samps_class, counted_value, count_index, st, endd = count_classes(df_test2)

    tot = [endd[i]-st[i] for i in range(len(st))]
    val = min(tot)

    # Ensurez que X est le meme taille
    X = []
    Y = []
    for i in range(len(tot)):
        isamp = endd[i]-st[i]
        diff = isamp - val
        X.append(df_test2.iloc[st[i]:endd[i]-diff, X_cols].to_numpy())
        Y.append(df_test2.iloc[st[i]:endd[i]-diff, y_cols].to_numpy())

    # ----------------

    X = np.array(X)
    
    # Tensorflow says to use :
    batch, timesteps, feature = X.shape

    print('batch:' , batch)
    print('timesteps:' , timesteps)
    print('feature:' , feature)
    
    print('taille de X:' , X.shape)
    # X.shape =  (104570, 20, 1)   # batch, timesteps/sequence length, feature

    # ----------------

    Y = np.array(Y)
    batch, timesteps, n_outputs = Y.shape   # batch, timesteps, 1
    print('taille de Y:' , Y.shape)

    # ----------------

    # Split the X an y data into test and train
    seed = 0
    test_size = 0.25 # default

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = seed, test_size = test_size)

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    
    print('X_train:' , X_train.shape)
    print('Y_train:' , Y_train.shape)
    
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)
    print('X_test:' , X_test.shape)
    print('Y_test:' , Y_test.shape)
    
    # ----------------
    
    # Ensure that Y_train and Y_test are integers
    Y_test = Y_test.astype(int)
    Y_train = Y_train.astype(int)
    
    # ----------------
    
    # batch_train, timesteps_train, feature_train = X_train.shape
    # batch_test, timesteps_test, feature_test = X_test.shape
    
    # OU
    
    suf = ['train', 'test']
    noms = ['batch_', 'timesteps_', 'feature_']
    dictkeys = [j+i for i in suf for j in noms]
    #print('dictkeys: ', dictkeys)

    dictvals = []
    dictvals.append(list(X_train.shape))
    dictvals.append(list(X_test.shape))
    dictvals = np.ravel(dictvals)
    
    info = dict(zip(dictkeys, dictvals))
    info['n_outputs'] = n_outputs
    # ----------------
    
    return X_train, X_test, Y_train, Y_test, info

# SVM

In [ ]:
def svm_batch(X_train, X_test, Y_train, Y_test, info, batch_size):
    
    # ----------------------------
    
    # batch_size = 24
    which_mod = 'SVC'  # 'NuSVC', 'SVC'
    para = 'adaptive' # 'hyperparmetre', 'adaptive'

    # ----------------------------

    class_len = len(np.unique(Y_train))
    if class_len <= 2:
        dfs = 'binary'
        dfs_var = 'ovo'
    elif class_len > 2:
        dfs = 'multi'
        dfs_var = 'ovr'

    # ----------------------------

    testall = 'batch'
    if testall == 'batch':
        # Test on same batch for all X_train
        st = np.random.permutation(int(np.ceil(info['batch_test']/batch_size)))[0]*batch_size
        endd = st+batch_size

        if endd < info['batch_test']:
            X_test_batch = np.reshape(X_test[st:endd,:,:], (batch_size*info['timesteps_test'], info['feature_test']))
            Y_test_1D_batch = np.reshape(Y_test[st:endd,:,:], (batch_size*info['timesteps_test'], info['n_outputs']))
        else:
            batch_mod = info['batch_test']-st
            endd = st+batch_mod
            X_test_batch = np.reshape(X_test[st:endd,:,:], (batch_mod*info['timesteps_test'], info['feature_test']))
            Y_test_1D_batch = np.reshape(Y_test[st:endd,:,:], (batch_mod*info['timesteps_test'], info['n_outputs']))
    elif testall == 'toutes_donnes':
        # Toutes des donnes
        X_test_batch = np.reshape(X_test, (info['batch_test']*info['timesteps_test'], info['feature_test']))
        Y_test_1D_batch = np.reshape(Y_test, (info['batch_test']*info['timesteps_test'], info['n_outputs']))

    n = int(np.ceil(info['batch_train']/batch_size))
    print('n: ', n)

    # ----------------------------

    C_l = np.linspace(0.9, 2, n)
    gamma_l = np.linspace(1/info['feature_train'], 1/2, n)

    C = 1  # Defaut
    gamma = 1/info['feature_train']
    inc_C = (0.9)*(1/10)
    inc_gamma = (1/info['feature_train'])*(1/10)
    # ----------------------------


    results = []

    for i in range(n):
        st = i*batch_size
        endd = st+batch_size

        # Stack all data over batches
        if endd < info['batch_train']:
            X_train_batch = np.reshape(X_train[st:endd,:,:], (batch_size*info['timesteps_train'], info['feature_train']))
            Y_train_1D_batch = np.reshape(Y_train[st:endd,:,:], (batch_size*info['timesteps_train'], info['n_outputs']))
        else:
            batch_mod = info['batch_train']-st
            endd = st+batch_mod
            X_train_batch = np.reshape(X_train[st:endd,:,:], (batch_mod*info['timesteps_train'], info['feature_train']))
            Y_train_1D_batch = np.reshape(Y_train[st:endd,:,:], (batch_mod*info['timesteps_train'], info['n_outputs']))

        # print('shape of X_train_batch : ', X_train_batch.shape)
        # print('shape of Y_train_1D_batch : ', Y_train_1D_batch.shape)

        # “one-versus-one” : binary ONLY, Y_train_1D, same implementation as libsvm 
        # (uses 1/lambda instead of C in cost function)
        if para == 'hyperparmetre':
            if which_mod == 'NuSVC':
                model = svm.NuSVC(decision_function_shape=dfs_var, gamma=gamma_l[i], probability=True, max_iter=-1)
            elif which_mod == 'SVC':
                model = svm.SVC(decision_function_shape=dfs_var, C=C_l[i], probability=True, max_iter=-1)
        elif para == 'adaptive':
            if which_mod == 'NuSVC':
                model = svm.NuSVC(decision_function_shape=dfs_var, gamma=gamma, probability=True, max_iter=-1)
            elif which_mod == 'SVC':
                model = svm.SVC(decision_function_shape=dfs_var, C=C, probability=True, max_iter=-1)

        model.fit(X_train_batch, Y_train_1D_batch)
        # ----------------------------

        Y_train_1D_predict = model.predict(X_train_batch)
        Y_test_1D_predict = model.predict(X_test_batch)

        # The prediction probability of each class : is size [n_samples, n_classes]
        Y_train_bin_pp = model.predict_proba(X_train_batch) 
        Y_test_bin_pp = model.predict_proba(X_test_batch)

        Y_train_bin_pp = np.array(Y_train_bin_pp)
        # print('shape of Y_train_bin_pp : ', Y_train_bin_pp.shape)
        Y_test_bin_pp = np.array(Y_test_bin_pp)
        # print('shape of Y_test_bin_pp : ', Y_test_bin_pp.shape)

        # How confidently each value predicted for x_test by the classifier is Positive ( large-magnitude Positive value ) or Negative ( large-magnitude Negative value)
        Y_train_score = model.decision_function(X_train_batch)  # size is [n_samples, 1]
        Y_test_score = model.decision_function(X_test_batch)

        Y_train_score = np.array(Y_train_score)
        Y_test_score = np.array(Y_test_score)

        # ----------------------------

        value_pack_train = evaluation_methods(model, X_train_batch, Y_train_1D_batch, 
                                              Y_train_1D_predict, Y_train_bin_pp, Y_train_score, dfs)
        value_pack_test = evaluation_methods(model, X_test_batch, Y_test_1D_batch, 
                                              Y_test_1D_predict, Y_test_bin_pp, Y_test_score, dfs)

        # ----------------------------

        if i > 0:
            acc_test_prev = acc_test

        # ----------------------------

        acc_train, prec_train, recall_train, roc_auc_train = mettez_dictout_dans_vars(value_pack_train, q=2)
        acc_test, prec_test, recall_test, roc_auc_test = mettez_dictout_dans_vars(value_pack_test, q=3) 

        # ----------------------------

        if which_mod == 'SVC':
            if i > 0:
                if acc_test_prev > acc_test:
                    C = C - inc_C
                else:
                    C = C + inc_C
            results.append([i, C, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test])

        elif which_mod == 'NuSVC':
            if i > 0:
                if acc_test_prev > acc_test:
                    gamma = gamma + inc_gamma
                else:
                    gamma = gamma - inc_gamma
            results.append([i, gamma, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test])

        # ----------------------------


        # Save model to file
        file_name = "model_%d.pkl" % (i)
        save_dat_pickle(model, file_name=file_name)

        # Delete model
        del model
        del Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_score, Y_test_score


    # Evaluatez quel modeles est mieux : AUCROC_test > 0.5 et max accuracy
    results = np.array(results)

    df = pd.DataFrame(results)
    df2 = df[(df.iloc[:,9] > 0.5)]
    best = df2.iloc[:,6].idxmax()
    print('best : ', best)
    print('C ou gamma : ', df.iloc[best,1])

    acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test = map(float, results[best,2::])

    # Load best model only for permutation importance
    file_name = "model_%d.pkl" % (best)
    model = load_dat_pickle(file_name=file_name)

    # Delete all .pkl files
    # del results
    rm_list = [j for j in range(n) if j != best]
    for i in rm_list:
        os.remove("model_%d.pkl" % (i))
    
    return model, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test

In [ ]:
def transform_Y_bin_pp_2_Y_1D_pp(Y_1D, Y_bin_pp):

    # Y_bin_pp is size [n_samples, n_classes=2]
    # Take the column of Y_bin_pp for the class of Y_1D, because both vectors need to be [n_samples, 1]
    Y_1D_pp = []
    for q in range(len(Y_1D)):
        desrow = Y_bin_pp[q]
        Y_1D_pp.append(desrow[int(Y_1D[q])])
    Y_1D_pp = np.ravel(Y_1D_pp)
    
    Y_1D_pp = np.array(Y_1D_pp)
    
    return Y_1D_pp

In [ ]:
def evaluation_methods(model, X, Y_1D, Y_1D_predict, Y_bin_pp, Y_score, dfs):
    
    average = 'micro'
    acc_dircalc = metrics.accuracy_score(Y_1D, Y_1D_predict)
    prec_dircalc = metrics.precision_score(Y_1D, Y_1D_predict, average=average)
    recall_dircalc = metrics.recall_score(Y_1D, Y_1D_predict, average=average)
    f1_dircalc = metrics.f1_score(Y_1D, Y_1D_predict, average=average)
    
    if dfs == 'binary':
        # ----------------------------
        Y_1D_pp = transform_Y_bin_pp_2_Y_1D_pp(Y_1D, Y_bin_pp)
        Y_1D = np.array(Y_1D)
        # ----------------------------
        
        # prediction probability
        rocauc_dircalc = metrics.roc_auc_score(Y_1D, Y_1D_pp, average=average)
        
    elif dfs == 'multi':
    
        # ----------------------------
        # Need to binarize Y into size [n_samples, n_classes]
        Y_bin, unique_classes = binarize_Y1Dvec_2_Ybin(Y_1D)
        Y_bin = np.array(Y_bin)
        # ----------------------------
        
        # decision function
        rocauc_dircalc = metrics.roc_auc_score(Y_bin, Y_score, average=average)

    
    value_pack = {}
    var_list = ['acc_dircalc', 'prec_dircalc', 'recall_dircalc', 'f1_dircalc', 'rocauc_dircalc']
    var_list_num = [acc_dircalc, prec_dircalc, recall_dircalc, f1_dircalc, rocauc_dircalc]
    
    for q in range(len(var_list)):
        value_pack['%s' % (var_list[q])] = var_list_num[q]
    
    return value_pack

# Random Forest

In [ ]:
def RandomForest(X_train, X_test, Y_train_1D, Y_test_1D):
    
    # Determine if classes are binary or multiclass:
    class_len = len(np.unique(Y_train_1D))
    if class_len <= 2:
        dfs = 'binary'
    elif class_len > 2:
        dfs = 'multi'
    
    forest = RandomForestClassifier(random_state=1, min_samples_leaf=50)  # min_samples_leaf is 100 by default
    model = MultiOutputClassifier(forest, n_jobs=-1) #n_jobs=-1 means apply parallel processing
    
    Y_train_1D = np.reshape(Y_train_1D, (len(Y_train_1D), 1))  # Y needs to have a defined shape ***
    model.fit(X_train, Y_train_1D)

    # ------------------------------
    
    Y_train_1D_predict = model.predict(X_train)
    Y_test_1D_predict = model.predict(X_test)

    # ------------------------------
    
    # Binary : the prediction probability of each class : is size [n_samples, n_classes]
    # Multi-class : the prediction probability of each class : size is [1, n_samples, n_classes]
    Y_train_bin_pp = model.predict_proba(X_train) 
    Y_test_bin_pp = model.predict_proba(X_test)
    
    if dfs == 'binary':
        Y_train_bin_pp = np.reshape(Y_train_bin_pp, (len(Y_train_1D_predict), 2))
        Y_test_bin_pp = np.reshape(Y_test_bin_pp, (len(Y_test_1D_predict), 2))
    elif dfs == 'multi':
        unique_classes = np.unique(Y_train_1D)
        Y_train_bin_pp = np.reshape(Y_train_bin_pp, (len(Y_train_1D), len(unique_classes)))
        Y_test_bin_pp = np.reshape(Y_test_bin_pp, (len(Y_test_1D), len(unique_classes)))
    
    # ------------------------------
    
    # There is NO decision_function
    # ------------------------------
    if dfs == 'binary':
        # size is [n_samples, 1]
        Y_train_score = transform_Y_bin_pp_2_Y_1D_pp(Y_train_1D, Y_train_bin_pp)
        Y_test_score = transform_Y_bin_pp_2_Y_1D_pp(Y_test_1D, Y_test_bin_pp)
        # OR
        # How confidently each value predicted for x_test by the classifier is Positive ( large-magnitude Positive value ) or Negative ( large-magnitude Negative value)
        #Y_train_1D_score = model.decision_function(X_train)  # size is [n_samples, 1]
        #Y_test_1D_score = model.decision_function(X_test)
    elif dfs == 'multi':
        # size is [n_samples, n_classes]
        Y_train_score = Y_train_bin_pp
        Y_test_score = Y_test_bin_pp
        
    Y_train_score = np.array(Y_train_score)
    Y_test_score = np.array(Y_test_score)
    
    # ------------------------------

    return model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_score, Y_test_score, dfs

# CNN

In [ ]:
def initialize_CNN(df_test2, X_cols, y_cols, img_dim): 
    
    X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols)
    # X_train: (batch_train, timesteps_train, feature_train)
    # Y_train: (batch_train, timesteps_train, n_outputs)
    # X_test: (batch_test, timesteps_test, feature_train)
    # Y_test: (batch_test, timesteps_test, n_outputs)
    
    # ----------------
    
    # Tranformez X(batch, timestamps, feature) into X(batch, img_dim, img_dim, 3)
    X_train_img = Xbtf_2_Xbii3(X_train, img_dim, info['batch_train'])
    X_test_img = Xbtf_2_Xbii3(X_test, img_dim, info['batch_test'])
    
    # ----------------
    
    Y_train_1D =  [Y_train[i,0:1,0] for i in range(info['batch_train'])]
    Y_test_1D =  [Y_test[i,0:1,0] for i in range(info['batch_test'])]
    Y_train_1D = np.array(Y_train_1D)
    Y_test_1D = np.array(Y_test_1D)
    
    # ----------------
    
    # shape of X_train_img :  (batch_train, img_dim, img_dim, 3)
    # shape of Y_train_1D :  (batch_train,1)
    # shape of X_test_img :  (batch_test, img_dim, img_dim, 3)
    # shape of Y_test_1D :  (batch_test,)
    
    X_train_img = np.asarray(X_train_img, dtype = np.float16, order ='C')  # np.float16, np.float32, np.float64
    Y_train_1D = np.asarray(Y_train_1D, dtype = np.float16, order ='C')
    X_test_img = np.asarray(X_test_img, dtype = np.float16, order ='C')
    Y_test_1D = np.asarray(Y_test_1D, dtype = np.float16, order ='C')
    
    print('X_train_img:' , X_train_img.shape)
    print('Y_train_1D:' , Y_train_1D.shape)
    print('X_test_img:' , X_test_img.shape)
    print('Y_test_1D:' , Y_test_1D.shape)
    
    return X_train_img, X_test_img, Y_train_1D, Y_test_1D, info
    
    
# Tranformez X(batch, timestamps, feature) into X(batch, img_dim, img_dim, 3)
def Xbtf_2_Xbii3(X, img_dim, batch):
    X_img = []
    for i in range(batch):
        X_1D = X[i,:,:].flatten()
        
        if i == 0:
            n = int(np.floor(np.sqrt(len(X_1D))))
    
        # fold into a square
        mat = np.reshape(X_1D[0:n*n], (n, n))
    
        image = imgORmat_resize_imgORmat_CNN(img_dim, mat, inpt='mat2D', outpt='img3D', norm='non', thresh='non')
        
        X_img.append(image)
    
    X_img = np.array(X_img)
    
    return X_img

In [ ]:
# 'Though for the 1D signal classification CNNs are also suitable, like they are implemented in the article 
# [49] for the seismic signal classification, while dealing with 2D objects CNNs can perform significantly 
# better results. Thus, firstly, we convert the 1D accelerometer signal into the 2D images via applying 
# CWT in order to extract signal features and, at the same time, to make it possible to implement 2D CNNs.'

def MPCNN_arch(n_outputs, img_dim, rgb_layers, ynum):
    
    # Typical architecture MPCNN architecture using alternating convolutional and max-pooling layers. 
    
    model = Sequential()  # initialize Sequential model
    
    mod = 0
    if mod == 0:
        model.add(Conv2D(32, (5,5), strides=(1,1), padding='same', input_shape=(img_dim, img_dim, rgb_layers)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
        model.add(Conv2D(32 * 2, (5,5), strides=(1,1), padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    elif mod == 1:
        model.add(Conv2D(8,(4,4), strides=(1,1), padding='same', input_shape=(img_dim, img_dim, rgb_layers)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D((8,8), strides=(8,8), padding='same'))
        model.add(Conv2D(16,(2,2), strides=(1,1), padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D((4,4), strides=(4,4), padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(1000, activation='relu'))

    initializer = tf.keras.initializers.HeUniform()
    # initializer = tf.keras.initializers.HeNormal()
    # initializer = tf.keras.initializers.GlorotUniform()
    if ynum == 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))
    
    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 1)
    # --------
    
    # Compile the model for training
    opt = keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    if ynum == 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    
    # model.summary()
    
    return model

In [ ]:
def encoderdecoder_arch(n_outputs, img_dim, rgb_layers, ynum):
    
    base_dimension = 64          
    
    model = Sequential()
    # 1ère valeur (filters) : le nombre de tranches "(kernel_val,kernel_val)" qui composent l'image de sortie
    # 2eme valeur (kernel_size) : la taille de la carre/filtre que on glisse au dessous l'image 
    # 3eme valeur (stride): Le plus grande le stride valeur le plus petite l'image sortie : on prends z_dim/stride_num
    
    # --------
    # Entrée = (img_dim, img_dim, 1)
    model.add(Conv2D(base_dimension, (5,5), strides=(2,2), padding='same', input_shape=(img_dim, img_dim, rgb_layers)))
    print('model.output_shape :', model.output_shape)
    # Sortie = 
    # taille_sortie = (28 + 2*p - 5)/2 + 1

    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    # --------

    # --------
    # Entrée = 
    model.add(Conv2D(base_dimension * 2, (5,5), strides=(2,2), padding='same'))
    print('model.output_shape :', model.output_shape)
    # Sortie = 

    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    # --------

    # --------
    model.add(Flatten())

    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 4096)
    # --------
    
    initializer = tf.keras.initializers.HeUniform()
    # initializer = tf.keras.initializers.HeNormal()
    # initializer = tf.keras.initializers.GlorotUniform()
    if ynum == 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))
    
    print('model.output_shape :', model.output_shape)
    # model.output_shape : (None, 1)
    # --------
    
    # Compile the model for training
    opt = keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    if ynum == 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    
    # model.summary()
    
    return model

In [ ]:
def run_CNN(df_test2, X_cols, y_cols, ynum):

    # ----------------
    img_dim = 64
    
    # Folding data into CNN image format:
    X_train_img, X_test_img, Y_train_1D, Y_test_1D, info  = initialize_CNN(df_test2, X_cols, y_cols, img_dim)
    batch = info['batch_train']
    timesteps_train = info['timesteps_train'] 
    feature = info['feature_train']
    n_outputs = info['n_outputs']
    
    # ----------------
    
    # Model architecture
    epochs = 100
    batch_size = 32
    
    tot = []
    tot_mod = []
    mod_type = ['mpcnn', 'dcgan', 'encdec'] # CNN model architecture type
    
    for i in range(3):
        if i == 0:
            model = MPCNN_arch(n_outputs, img_dim, ynum)
        elif i == 1:
            den_activation = LeakyReLU(alpha=0.2)
            model = dcgan_arch(n_outputs, img_dim, ynum, den_activation)
        elif i == 2:
            model = encoderdecoder_arch(n_outputs, img_dim, ynum)
    
        patience = 5 # Number of epochs with no improvement after which training will be stopped.
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')
        
        # -------------------------------
        
        if i == 1:
            X_train_1D = []
            for lay in range(X_train_img.shape[0]):
                # Transformez 3D image à 2D matrix
                # Train
                X_train_2D = imgORmat_resize_imgORmat_CNN(img_dim, X_train_img[lay,:,:,:], inpt='img3D', outpt='mat2D', norm='non', thresh='non')
                X_train_1D.append(X_train_2D.flatten())
            
            X_test_1D = []
            for lay in range(X_test_img.shape[0]):
                # Test
                X_test_2D = imgORmat_resize_imgORmat_CNN(img_dim, X_test_img[lay,:,:,:], inpt='img3D', outpt='mat2D', norm='non', thresh='non')
                X_test_1D.append(X_test_2D.flatten())
            
            history = model.fit(X_train_1D, Y_train_1D, epochs=epochs, validation_data=(X_test_1D, Y_test_1D), batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        else:
            history = model.fit(X_train_img, Y_train_1D, epochs=epochs, validation_data=(X_test_img, Y_test_1D), batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        
        
        history_df = pd.DataFrame(history.history)
        out = [history_df.iloc[:,i].mean() for i in range(len(history_df.columns))]
    
        tot.append(out)
        tot_mod.append(model)
    
    tot = np.array(tot)
    
    # -------------------------------
    
    a = np.argmax(tot[:,1])  # train
    b = np.argmax(tot[:,6])  # test
    suf = ['train', 'test']
    tr_noms = ['loss_', 'acc_', 'prec_', 'recall_', 'roc_auc_']

    list2 = [j+i for i in suf for j in tr_noms]
    list2

    dict_out = {}
    for i in range(len(list2)):
        if i < len(list2)/2:
            r = tot[a,i]
        else:
            r = tot[b,i]
        dict_out[list2[i]] = r


    # ajoutez au dictionaire
    dict_out['mod_train'] = mod_type[a]
    dict_out['mod_test'] = mod_type[b]

    # -------------------------------

    cnn2D_model_best = tot_mod[b]
    
    # -------------------------------
    
    return cnn2D_model_best, dict_out, X_test, Y_test

# LSTM

In [ ]:
def LSTM_arch(n_a, timesteps_train, feature, return_sequences, return_state, stateful, n_outputs, ynum):
    
    model = Sequential()

    if stateful == True:
        # Quand vous definez batch_input_shape, il faut que des entries de modele être le meme taille que le train dataset
        model.add(LSTM(n_a, input_shape=(timesteps_train, feature), batch_input_shape=(batch, timesteps_train, feature), return_sequences=return_sequences, return_state=return_state, stateful=stateful))
    elif stateful == False:
        model.add(LSTM(n_a, input_shape=(timesteps_train, feature), return_sequences=return_sequences, return_state=return_state, stateful=stateful))

    # Types of W initializer :
    initializer = tf.keras.initializers.HeUniform()

    if ynum == 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))

    # Compile the model for training
    # opt = keras.optimizers.Adam()
    opt = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
    # opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

    # Si vous utilisez softmax activation, la taille de sortie est plus grand que deux donc il faut categorical_crossentropy
    if ynum == 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'

    # model.summary()

    return model

In [ ]:
def initialize_LSTM(df_test2, X_cols, y_cols, return_sequences, stateless):
    
    X_train, X_test, Y_train, Y_test, info = df_2_XYtraintest_formatbatch_timestep_feature(df_test2, X_cols, y_cols)
    
    # ----------------
    
    tf_train = X_Y_a_tfobj(X_train, Y_train, info['batch_train'], return_sequences, stateless)
    tf_test = X_Y_a_tfobj(X_test, Y_test, info['batch_test'], return_sequences, stateless)
    
    # side idea : if I give tf_test info['batch_train'] instead of info['batch_test'], could I run stateful=True
    # for all data (train and test)?
    # ----------------

    return tf_train, tf_test, X_train, Y_train, X_test, Y_test, info



def X_Y_a_tfobj(X, Y, batch_size, return_sequences, stateless):
    
    if return_sequences == False:  # one output per batch of samples
        # Y shape : batch_size,      ie: Y.shape =  (104570,)
        temp = [Y[i,0:1,:] for i in range(Y.shape[0])]
        Y = np.array(temp)
        Y = np.reshape(Y, (batch_size,))
    elif return_sequences == True: # an output per each batch of samples
        # Y shape : batch_size, timesteps, n_output
        Y = np.array(Y)
    
    # ----------------
    
    tf_data = tf.data.Dataset.from_tensor_slices((X, Y))
    
    if stateless == True:  # sequence is non-causal
        buffer_size = 1000
        tf_data = tf_data.cache().shuffle(buffer_size).batch(batch_size)
    elif stateless == False:  # sequence is causal
        tf_data = tf_data.batch(batch_size)
    
    return tf_data

In [ ]:
def run_LSTM(df_test2, X_cols, y_cols, ynum):
    
    return_sequences = False # True=return a prediction at every batch sample, (default) False=return one prediction at the end of the batch
    stateless = True # True=shuffle the batch samples/slices --samples are non-causal, False=do not shuffle slices---samples are causal
    
    # -------------------------------
    
    # Folding data into LSTM format:
    tf_train, tf_test, X_train, Y_train, X_test, Y_test, info  = initialize_LSTM(df_test2, X_cols, y_cols, return_sequences, stateless)
    batch = info['batch_train']
    timesteps_train = info['timesteps_train'] 
    feature = info['feature_train']
    n_outputs = info['n_outputs']
    
    # -------------------------------

    # Model architecture
    epochs = 100
    batch_size = 32
    return_state = False # True=return a and c, (default) False=do not return hidden state (a) and cell state (c)
    stateful = False #  If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.
    
    tot = []
    tot_mod = []
    liste_de_vals = [20, 40, 60, 70, 80, 100] # number of dimensions for the hidden state of each LSTM cell
    
    for n_a in liste_de_vals:

        model = LSTM_arch(n_a, timesteps_train, feature, return_sequences, return_state, stateful, n_outputs, ynum)
        # -------------------------------
        
        # 50 est trop, 20 est insuffisant 
        patience = 28 # Number of epochs with no improvement after which training will be stopped.
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')
        
        # -------------------------------

        if stateful == False:
            history = model.fit(tf_train, epochs=epochs, validation_data=tf_test, batch_size=batch_size, callbacks=[early_stopping], verbose=0)
            # OU
            # history = model.fit(tf_train, epochs=epochs, batch_size=batch_size, verbose=0)
            # score = model.evaluate(tf_test, verbose=2)
            # test_acc.append(score[1])
            # test_loss.append(score[0])
        elif stateful == True:
            # On peut faire tf_train parce que le taille est fixer at batch_input_shape
            history = model.fit(tf_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=0)
    
        
        history_df = pd.DataFrame(history.history)
        out = [history_df.iloc[:,i].mean() for i in range(len(history_df.columns))]
        
        tot.append(out)
        tot_mod.append(model)
    
    tot = np.array(tot)
    
    # -------------------------------
    
    a = np.argmax(tot[:,1])  # train
    b = np.argmax(tot[:,6])  # test
    suf = ['train', 'test']
    tr_noms = ['loss_', 'acc_', 'prec_', 'recall_', 'roc_auc_']

    list2 = [j+i for i in suf for j in tr_noms]
    list2

    dict_out = {}
    for i in range(len(list2)):
        if i < len(list2)/2:
            r = tot[a,i]
        else:
            r = tot[b,i]
        dict_out[list2[i]] = r


    # ajoutez au dictionaire
    dict_out['delay_train'] = liste_de_vals[a]
    dict_out['delay_test'] = liste_de_vals[b]

    # -------------------------------

    lstm_model_best = tot_mod[b]
    
    # -------------------------------
    
    return lstm_model_best, dict_out, X_test, Y_test

# LSTM-CNN

In [ ]:
def run_LSTMCNN(df_test2, X_cols, y_cols, ynum):
    
    return_sequences = False # True=return a prediction at every batch sample, (default) False=return one prediction at the end of the batch
    stateless = True # True=shuffle the batch samples/slices --samples are non-causal, False=do not shuffle slices---samples are causal
    
    # -------------------------------
    
    # Folding data into LSTM format:
    tf_train, tf_test, X_train, Y_train, X_test, Y_test, info  = initialize_LSTM(df_test2, X_cols, y_cols, return_sequences, stateless)
    batch = info['batch_train']
    timesteps_train = info['timesteps_train'] 
    feature = info['feature_train']
    
    
    # -------------------------------

    # Model architecture 1
    epochs = 100
    batch_size = 32
    return_state = False # True=return a and c, (default) False=do not return hidden state (a) and cell state (c)
    stateful = False #  If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.
    
    n_a = 32  # 20, 40
    n_outputs = n_a
    model = LSTM_arch(n_a, timesteps_train, feature, return_sequences, return_state, stateful, n_outputs, ynum)

    # -------------------------------

    # 50 est trop, 20 est insuffisant 
    patience = 28 # Number of epochs with no improvement after which training will be stopped.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')

    # -------------------------------

    if stateful == False:
        history = model.fit(tf_train, epochs=epochs, validation_data=tf_test, batch_size=batch_size, callbacks=[early_stopping], verbose=0)
    elif stateful == True:
        # On peut faire tf_train parce que le taille est fixer at batch_input_shape
        history = model.fit(tf_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=0)
        
    # -------------------------------

    lstm_model_best = model
    
    X_lstm_train = lstm_model_best.predict(X_train)
    X_lstm_test = lstm_model_best.predict(X_test)
    # should be (batch, n_a)
    
    X_lstm_train = np.array(X_lstm_train)
    X_lstm_test = np.array(X_lstm_test)
    
    print('X_lstm_train : ', X_lstm_train.shape)
    print('X_lstm_test : ', X_lstm_test.shape)
    
    
    
    
    # -------------------------------
    # CNN
    # -------------------------------
    img_dim = 64
    
    # Folding data into CNN image format:
    
    # Tranformez X(batch, timestamps, feature) into X(batch, img_dim, img_dim, 3)
    X_train_img = Xbtf_2_Xbii3(X_lstm_train, img_dim, X_lstm_train.shape[0])
    X_test_img = Xbtf_2_Xbii3(X_lstm_test, img_dim, X_lstm_test.shape[0])
    
    # ----------------
    
    Y_train_1D =  [Y_train[i,0:1,0] for i in range(Y_train.shape[0])]
    Y_test_1D =  [Y_test[i,0:1,0] for i in range(Y_test.shape[0])]
    Y_train_1D = np.array(Y_train_1D)
    Y_test_1D = np.array(Y_test_1D)
    
    # ----------------
    
    # shape of X_train_img :  (batch_train, img_dim, img_dim, 3)
    # shape of Y_train_1D :  (batch_train,1)
    # shape of X_test_img :  (batch_test, img_dim, img_dim, 3)
    # shape of Y_test_1D :  (batch_test,)
    
    X_train_img = np.asarray(X_train_img, dtype = np.float16, order ='C')  # np.float16, np.float32, np.float64
    Y_train_1D = np.asarray(Y_train_1D, dtype = np.float16, order ='C')
    X_test_img = np.asarray(X_test_img, dtype = np.float16, order ='C')
    Y_test_1D = np.asarray(Y_test_1D, dtype = np.float16, order ='C')
    
    print('X_train_img:' , X_train_img.shape)
    print('Y_train_1D:' , Y_train_1D.shape)
    print('X_test_img:' , X_test_img.shape)
    print('Y_test_1D:' , Y_test_1D.shape)
    
    
    # ----------------
    
    # Model architecture 2
    epochs = 100
    batch_size = 32
    
    tot = []
    tot_mod = []
    mod_type = ['mpcnn', 'dcgan', 'encdec'] # CNN model architecture type
    
    for i in range(3):
        if i == 0:
            model = MPCNN_arch(n_outputs, img_dim, ynum)
        elif i == 1:
            model = dcgan_arch(n_outputs, input_shape, ynum)
        elif i == 2:
            model = encoderdecoder_arch(n_outputs, img_dim, ynum)
    
        patience = 5 # Number of epochs with no improvement after which training will be stopped.
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')
        
        # -------------------------------
        
        if i == 1:
            X_train_1D = []
            for lay in range(X_train_img.shape[0]):
                # Transformez 3D image à 2D matrix
                # Train
                X_train_2D = imgORmat_resize_imgORmat_CNN(img_dim, X_train_img[lay,:,:,:], inpt='img3D', outpt='mat2D', norm='non', thresh='non')
                X_train_1D.append(X_train_2D.flatten())
            
            X_test_1D = []
            for lay in range(X_test_img.shape[0]):
                # Test
                X_test_2D = imgORmat_resize_imgORmat_CNN(img_dim, X_test_img[lay,:,:,:], inpt='img3D', outpt='mat2D', norm='non', thresh='non')
                X_test_1D.append(X_test_2D.flatten())
            
            history = model.fit(X_train_1D, Y_train_1D, epochs=epochs, validation_data=(X_test_1D, Y_test_1D), batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        else:
            history = model.fit(X_train_img, Y_train_1D, epochs=epochs, validation_data=(X_test_img, Y_test_1D), batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        
        
        history_df = pd.DataFrame(history.history)
        out = [history_df.iloc[:,i].mean() for i in range(len(history_df.columns))]
    
        tot.append(out)
        tot_mod.append(model)
    
    tot = np.array(tot)
    
    # -------------------------------
    
    a = np.argmax(tot[:,1])  # train
    b = np.argmax(tot[:,6])  # test
    suf = ['train', 'test']
    tr_noms = ['loss_', 'acc_', 'prec_', 'recall_', 'roc_auc_']

    list2 = [j+i for i in suf for j in tr_noms]
    list2

    dict_out = {}
    for i in range(len(list2)):
        if i < len(list2)/2:
            r = tot[a,i]
        else:
            r = tot[b,i]
        dict_out[list2[i]] = r


    # ajoutez au dictionaire
    dict_out['mod_train'] = mod_type[a]
    dict_out['mod_test'] = mod_type[b]

    # -------------------------------

    cnn2D_model_best = tot_mod[b]
    
    # -------------------------------
    
    return cnn2D_model_best, dict_out, X_test, Y_test

# Transformer

In [ ]:
def positional_encoding(positions, d):
    """
    Precomputes a matrix with all the positional encodings 
    
    Arguments:
        positions (int) -- Maximum number of positions to be encoded 
        d (int) -- Encoding size 
    
    Returns:
        pos_encoding -- (1, position, d_model) A matrix with the positional encodings
    """
    # START CODE HERE
    # initialize a matrix angle_rads of all the angles
    # Get the angles for the positional encoding
    # pos -- Column vector containing the positions [[0], [1], ...,[N-1]]
    # i --   Row vector containing the dimension span [[0, 1, 2, ..., M-1]]
    pos = np.reshape(np.multiply(range(0, positions), 1), (positions,1))
    i = np.reshape(np.multiply(range(0, d), 1), (1,d))
    angle_rads = pos / (10000 ** ((2*(i//2))/d))
    
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    # END CODE HERE
    
    pos_encoding = angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
# Padding Mask
# Sequences longer than the maximum length of five will be truncated, and zeros will be added to the 
# truncated sequence to achieve uniform length. Similarly, for sequences shorter than the maximum 
# length, they zeros will also be added for padding. 

def create_padding_mask(seq):
    """
    Creates a matrix mask for the padding cells
    
    Arguments:
        seq -- (n, m) matrix
    
    Returns:
        mask -- (n, 1, 1, m) binary tensor
    """
    # If x equals 0 it put a True, and if not puts a False
    #out1 = tf.math.equal(x, 0)  
    #print('out1 : ' + str(out1))

    # Converts the boolean matrix to a float32
    #out2 = tf.cast(out1, tf.float32)
    #print('out2 : ' + str(out2))
    
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
    # add extra dimensions to add the padding to the attention logits.
    # it takes the (3,5) sized seq matrix and makes it a (3, 1, 1, 5) matrix
    return seq[:, tf.newaxis, tf.newaxis, :] 

In [ ]:
# Look-ahead Mask
# In training, you will have access to the complete correct output of your training example. 
# The look-ahead mask helps your model pretend that it correctly predicted a part of the output and 
# see if, without looking ahead, it can correctly predict the next output.

def create_look_ahead_mask(size):
    """
    Returns an upper triangular matrix filled with ones
    
    Arguments:
        size -- matrix size
    
    Returns:
        mask -- (size, size) tensor
    """
    #Based on the lower and upper matrix diagonal keep values, it keeps 
    #the matrix entries and sets the other entries to zero
    #If lower_mat_dia_keep=0 and upper_mat_dia_keep=1 it keeps the diagonal and one upper
    #diagonal section, similarly lower_mat_dia_keep=-1 and upper_mat_dia_keep=0 it keeps
    #one lower diagonal section and the diagonal
    #out1 = tf.linalg.band_part(tf.ones((3, 3)), lower_mat_dia_keep, upper_mat_dia_keep)
    #print('out1 : ' + str(out1))
    
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

In [ ]:
def FullyConnected(embedding_dim, fully_connected_dim):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(fully_connected_dim, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(embedding_dim)  # (batch_size, seq_len, embedding_dim) = (batch, timesteps, feature)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    """
    The encoder layer is composed by a multi-head self-attention mechanism,
    followed by a simple, positionwise fully connected feed-forward network. 
    This archirecture includes a residual connection around each of the two 
    sub-layers, followed by layer normalization.
    """
    def __init__(self, embedding_dim, num_heads, fully_connected_dim, dropout_rate=0.1, layernorm_eps=1e-6):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)

        self.ffn = FullyConnected(embedding_dim=embedding_dim, fully_connected_dim=fully_connected_dim)

        self.layernorm1 = LayerNormalization(epsilon=layernorm_eps)
        self.layernorm2 = LayerNormalization(epsilon=layernorm_eps)

        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)
    
    def call(self, x, training, mask):
        """
        Forward pass for the Encoder Layer
        
        Arguments:
            x -- Tensor of shape (batch_size, input_seq_len, fully_connected_dim)
            training -- Boolean, set to true to activate
                        the training mode for dropout layers
            mask -- Boolean mask to ensure that the padding is not 
                    treated as part of the input
        Returns:
            out2 -- Tensor of shape (batch_size, input_seq_len, fully_connected_dim)
        """
        # START CODE HERE
        # calculate self-attention using mha(~1 line)
        attn_output = self.mha(query=x, value=x, key=x, attention_mask=mask)  # Self attention (batch_size, input_seq_len, fully_connected_dim)
        
        # apply dropout layer to the self-attention output (~1 line)
        attn_output = self.dropout1(attn_output, training=training)
        
        # apply layer normalization on sum of the input and the attention output to get the  
        # output of the multi-head attention layer (~1 line)
        out1 = self.layernorm1(tf.add(x, attn_output))  # (batch_size, input_seq_len, fully_connected_dim)

        # pass the output of the multi-head attention layer through a ffn (~1 line)
        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, fully_connected_dim)
        
        # apply dropout layer to ffn output (~1 line)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        # apply layer normalization on sum of the output from multi-head attention and ffn output to get the
        # output of the encoder layer (~1 line)
        out2 = self.layernorm2(tf.add(out1, ffn_output))  # (batch_size, input_seq_len, fully_connected_dim)
        # END CODE HERE
        
        return out2

In [ ]:
class Encoder(tf.keras.layers.Layer):
    """
    The entire Encoder starts by passing the input to an embedding layer 
    and using positional encoding to then pass the output through a stack of
    encoder Layers
        
    """   
    def __init__(self, num_layers, embedding_dim, num_heads, fully_connected_dim, input_vocab_size,
               maximum_position_encoding, dropout_rate=0.1, layernorm_eps=1e-6, textORnot='timeseries'):
        super(Encoder, self).__init__()
        
        self.textORnot = textORnot
        
        if self.textORnot == 'text':
            self.embedding_dim = embedding_dim
        
        self.num_layers = num_layers

        self.embedding = Embedding(input_vocab_size, self.embedding_dim)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.embedding_dim)

        self.enc_layers = [EncoderLayer(embedding_dim=self.embedding_dim,
                                        num_heads=num_heads,
                                        fully_connected_dim=fully_connected_dim,
                                        dropout_rate=dropout_rate,
                                        layernorm_eps=layernorm_eps) 
                           for _ in range(self.num_layers)]

        self.dropout = Dropout(dropout_rate)
        
    def call(self, x, training, mask):
        """
        Forward pass for the Encoder
        
        Arguments:
            x -- Tensor of shape (batch_size, input_seq_len)
            training -- Boolean, set to true to activate
                        the training mode for dropout layers
            mask -- Boolean mask to ensure that the padding is not 
                    treated as part of the input
        Returns:
            out2 -- Tensor of shape (batch_size, input_seq_len, fully_connected_dim)
        """

        seq_len = tf.shape(x)[1]
        
        if self.textORnot == 'text':
            # Text ONLY : Pass input through the Embedding layer
            x = self.embedding(x)  # (batch_size, input_seq_len, embedding_dim) = (batch, timesteps, feature)
            # print('shape of embedding : ', x.shape)
        
        # Scale embedding by multiplying it by the square root of the embedding dimension
        # Remember to cast the embedding dimension to data type tf.float32 before computing the square root.
        x *= tf.sqrt(tf.cast(self.embedding_dim, tf.float32))
        
        # Add the position encoding to embedding
        x += self.pos_encoding[:, :seq_len, :]
        
        # Pass the encoded embedding through a dropout layer
        x = self.dropout(x, training=training)
        
        #print('shape of x : ' + str(x.shape))
        #shape of x : (2, 3, 4) so loop over the first entry
        
        # Pass the output through the stack of encoding layers 
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, embedding_dim) = (batch, timesteps, feature)

In [ ]:
def reshapeY3D_a_Y1D(Y_bin):
    # Y_bin : (batch, timesteps, n_outputs) OU Y_bin : (batch, n_outputs)
    
    batch = Y_bin.shape[0]
    
    if len(Y_bin.shape) == 2:
        # Y_bin : (batch, n_outputs)
        temp = [np.argmax(Y_bin[i,:]) for i in range(batch)]   # (batch,)
    
    elif len(Y_bin.shape) == 3:
        # Y_bin : (batch, timesteps, n_outputs)
        temp = [np.argmax(Y_bin[i,0:1,:]) for i in range(batch)]   # (batch,)
    
    Y = np.array(temp)
    Y = np.reshape(Y, (batch,)) # Y : (batch, )
    
    return Y

In [ ]:
def run_Transformer(X_train, X_test, Y_train, Y_test, patience, batch_size, epochs):
    
    # X_train : (batch_train, timesteps_train, feature)
    # X_test : (batch_test, timesteps_test, feature)
    
    # Y_train : (batch_train, n_outputs)
    # Y_test : (batch_test, n_outputs)
    
    # -------------------------------
    
    batch_train = X_train.shape[0]
    timesteps_train = X_train.shape[1]
    feature = X_train.shape[2]
    
    batch_test = X_test.shape[0]
    timesteps_test = X_test.shape[1]
    
    n_outputs = Y_train.shape[1]
    
    # -------------------------------
    
    # Model architecture
    
    # Encoder (shuffling and transformation of the data)
    tf.random.set_seed(1)  #10
    
    # unique words, or unique data points to a certain precision
    precision = 0.1
    input_vocab_size=len(np.arange(0,1,precision))
    
    # the maximum number of words in each sentence, OR timesteps
    maximum_position_encoding = timesteps_train
    
    encoderq = Encoder(num_layers=6, embedding_dim=feature, num_heads=1, fully_connected_dim=2*feature, 
                       input_vocab_size=input_vocab_size,
                       maximum_position_encoding=maximum_position_encoding, dropout_rate=0.1, 
                       layernorm_eps=1e-6, textORnot='timeseries')
    
    training = True  # training for Dropout layers
    mask = None #create_padding_mask(X_train)  # create_look_ahead_mask(x.shape[1]) # None
    encoder_X_train = encoderq(X_train, training, mask)   # output is shape=(batch_train, timesteps_train, feature)
    
    mask = None #create_padding_mask(X_test)  # create_look_ahead_mask(x.shape[1]) # None
    encoder_X_test = encoderq(X_test, training, mask)   # output is shape=(batch_test, timesteps_test, feature)
    
    # -------------------------------
    
    # Final Fully Connected
    model = Sequential()
    initializer = tf.keras.initializers.HeUniform()
    if n_outputs > 2:
        model.add(Dense(n_outputs, activation='softmax', kernel_initializer=initializer))
    else:
        model.add(Dense(n_outputs, activation='sigmoid', kernel_initializer=initializer))

    # --------

    # Compile the model for training
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    if n_outputs > 2:
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'
    else:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])  # optimizer='adam'

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')

    # -------------------------------
    
    X_train_1D = np.reshape(encoder_X_train, (batch_train, timesteps_train*feature))
    X_test_1D = np.reshape(encoder_X_test, (batch_test, timesteps_test*feature))
    
    verbose = 2
    #history = model.fit(X_train_1D, Y_train, epochs=epochs, 
    #                    validation_data=(X_test_1D, Y_test), 
    #                    batch_size=batch_size, callbacks=[early_stopping], verbose=verbose)
    history = model.fit(X_train_1D, Y_train, epochs=epochs, 
                        validation_data=(X_test_1D, Y_test), 
                        batch_size=batch_size, verbose=verbose)
    history_df = pd.DataFrame(history.history)
    
    # -------------------------------
    
    return model, history_df